In [1]:
from datetime import datetime
import numpy as np
import tensorflow as tf

In [2]:
n_inputs = 28*28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

In [3]:
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

In [4]:
def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2 / np.sqrt(n_inputs + n_neurons)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        W = tf.Variable(init, name="kernel")
        b = tf.Variable(tf.zeros([n_neurons]), name="bias")
        Z = tf.matmul(X, W) + b
        if activation is not None:
            return activation(Z)
        else:
            return Z

In [5]:
with tf.name_scope("dnn"):
    hidden1 = neuron_layer(X, n_hidden1, name="hidden1", activation=tf.nn.relu)
    hidden2 = neuron_layer(hidden1, n_hidden2, name="hidden2", activation=tf.nn.relu)
    logits = neuron_layer(hidden2, n_outputs, name="outputs")

In [6]:
# # 上二つのセルはこの記述と同意
# with tf.name_scope("dnn"):
#     hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1", activation=tf.nn.relu)
#     hidden2 = tf.layers.dense(hidden1, n_hidden2, name="hidden2", activation=tf.nn.relu)
#     logits = tf.layers.dense(hidden2, n_outputs, name="outputs")

In [7]:
# 損失の計算
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")
    
"""
logitsをソフトマックス関数に入力し、交差エントロピー(極小値の加算あり)を算出

tf.nn.sparse_softmax_cross_entropy_with_logits(labels, logits)
labels はクラスラベル [0,num_classes-1] のスカラー

tf.nn.softmax_cross_entropy_with_logits(labels, logits)
labels はクラスラベルのone-hot-vector
"""

'\nlogitsをソフトマックス関数に入力し、交差エントロピー(極小値の加算あり)を算出\n\ntf.nn.sparse_softmax_cross_entropy_with_logits(labels, logits)\nlabels はクラスラベル [0,num_classes-1] のスカラー\n\ntf.nn.softmax_cross_entropy_with_logits(labels, logits)\nlabels はクラスラベルのone-hot-vector\n'

In [8]:
# パラメータ更新
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [9]:
# 精度検証
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1) # 予測が正しいかどうかの論理値配列
    accuracy =  tf.reduce_mean(tf.cast(correct, tf.float32)) # 論理値を浮動小数点型にキャストして、平均の計算

In [10]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

# trainデータとvalidationデータでサマリーを分けたい場合は、以下のように保存する変数を変更する
with tf.name_scope("train_eval"):
    loss_summary_train = tf.summary.scalar('loss', loss)
    acc_summary_train = tf.summary.scalar('accuracy', accuracy)
    
with tf.name_scope("val_eval"):
    loss_summary_val = tf.summary.scalar('loss', loss)
    acc_summary_val = tf.summary.scalar('accuracy', accuracy)

# サマリーが複数ある場合はまとめることができるが、trainとvalidationとで入力データを分けているので、割愛
# merged_summary = tf.summary.merge_all()

In [11]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("~/tf_data/")

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ~/tf_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ~/tf_data/train-labels-idx1-ubyte.gz
Extracting ~/tf_data/t10k-images-idx3-ubyte.gz
Extracting ~/tf_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [12]:
n_epochs = 40
batch_size = 50

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
log_dir = "{}/run-{}/".format(root_logdir, now)

In [14]:
with tf.Session() as sess:
    init.run()
    
    with tf.summary.FileWriter(log_dir, tf.get_default_graph()) as file_writer:
        for epoch in range(n_epochs):
            for iteration in range(mnist.train.num_examples // batch_size):
                X_batch, y_batch = mnist.train.next_batch(batch_size)
                sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

                train_list = [acc_summary_train, loss_summary_train]
                loss_train, acc_train = sess.run(
                    train_list, # <-- サマリーをマージした場合は merged_summary を指定
                    feed_dict={X: X_batch, y: y_batch})
                file_writer.add_summary(loss_train, epoch)
                file_writer.add_summary(acc_train, epoch)
                
#                 result = sess.run(
#                     train_list, # <-- サマリーをマージした場合は merged_summary を指定
#                     feed_dict={X: X_batch, y: y_batch})
#                 for j in range(0,len(result)):
#                     file_writer.add_summary(result[j], epoch)

            val_list = [acc_summary_val, loss_summary_val]
            loss_val, acc_val = sess.run(
                val_list, # <-- サマリーをマージした場合は merged_summary を指定
                feed_dict={X: mnist.validation.images, y: mnist.validation.labels})
            file_writer.add_summary(loss_val, epoch)
            file_writer.add_summary(acc_val, epoch)

#             print(epoch, "Train accuracy :", acc_train, "Val accuracy :", acc_val)

        save_path = saver.save(sess, "./logs/dnn/my_model_final.ckpt")

0 Train accuracy : b'\n\x16\n\x0ftrain_eval/loss\x15X\x1d\xca>' Val accuracy : b'\n\x14\n\rval_eval/loss\x15\xe9\xcf\xa4>'
1 Train accuracy : b'\n\x16\n\x0ftrain_eval/loss\x15\x93G\x03>' Val accuracy : b'\n\x14\n\rval_eval/loss\x15\x10\x90\x81>'
2 Train accuracy : b'\n\x16\n\x0ftrain_eval/loss\x15\x9b\xc0\xe8=' Val accuracy : b'\n\x14\n\rval_eval/loss\x15\xd1\xe9]>'
3 Train accuracy : b'\n\x16\n\x0ftrain_eval/loss\x15\xbf\xdd{>' Val accuracy : b'\n\x14\n\rval_eval/loss\x15\xed}E>'
4 Train accuracy : b'\n\x16\n\x0ftrain_eval/loss\x15\x12\xf5\xea=' Val accuracy : b'\n\x14\n\rval_eval/loss\x15\xe0\xb93>'
5 Train accuracy : b'\n\x16\n\x0ftrain_eval/loss\x15\xd7\xc3y>' Val accuracy : b'\n\x14\n\rval_eval/loss\x15\x8c\x9d%>'
6 Train accuracy : b'\n\x16\n\x0ftrain_eval/loss\x15\x9b\x90\xe2=' Val accuracy : b'\n\x14\n\rval_eval/loss\x15\x03p\x1a>'
7 Train accuracy : b'\n\x16\n\x0ftrain_eval/loss\x15\x04}\x19>' Val accuracy : b'\n\x14\n\rval_eval/loss\x15\xe7S\x12>'
8 Train accuracy : b'\n\x16\

In [ ]:
# # 予測
# with tf.Session() as sess:
#     saver.restore(sess, "./DNN_ckpt/my_model_final.ckpt")
#     X_new_scaled = [...]
#     Z = logit.eval(feed_dcit={X: X_new_scaled})
#     y_pred = np.argmax(Z, axis=1)